In [1]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from multiprocessing.pool import ThreadPool
from sklearn.metrics import mean_squared_error
import math
import matplotlib.pyplot as plt
import numpy as np

In [2]:
data_match = pd.read_csv('/Users/swayam/Documents/Projects/Data_Mining/Data/Team_Strength_Match_Info.csv')
data_delivery = pd.read_csv('/Users/swayam/Documents/Projects/Data_Mining/Data/Delivery_Info.csv')

In [3]:
data_match = data_match.replace('Rising Pune Supergiants', 'Rising Pune Supergiant')
data_delivery = data_delivery.replace('Rising Pune Supergiants', 'Rising Pune Supergiant')
data_match = data_match.replace('Delhi Daredevils','Delhi Capitals')
data_delivery = data_delivery.replace('Delhi Daredevils','Delhi Capitals')
data_match = data_match.replace('Pune Warriors', 'Pune Warriors India')
data_delivery = data_delivery.replace('Pune Warriors', 'Pune Warriors India')

In [4]:
data_match.head(5)

,Match_ID,Season,Date,Team1,Team2,Toss_Winner,Toss_Decision,result,DL_Method,Winner,...,Man_Of_The_Match,Venue,Umpire1,Umpire2,Umpire3,Total_Balls,Team1_Strength,Team2_Strength,Team1_Strength.1,Team2_Strength.1
0,1,2013,5/19/13,Pune Warriors India,Delhi Capitals,Pune Warriors India,bat,Normal,0,Pune Warriors India,...,LJ Wright,Subrata Roy Sahara Stadium,NJ Llong,SJA Taufel,NaN,250,237.500000,242.250000,79.166667,20.471831
1,2,2015,4/15/15,Kings XI Punjab,Delhi Capitals,Kings XI Punjab,bat,Normal,0,Delhi Capitals,...,MA Agarwal,Maharashtra Cricket Association Stadium,CB Gaffaney,K Srinath,NaN,253,654.750000,1410.500000,100.730769,88.156250
2,3,2011,5/14/11,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Normal,1,Royal Challengers Bangalore,...,CH Gayle,M Chinnaswamy Stadium,RE Koertzen,RB Tiffin,NaN,159,579.000000,385.375000,28.016129,27.043860
3,4,2017,5/7/17,Kings XI Punjab,Gujarat Lions,Gujarat Lions,field,Normal,0,Gujarat Lions,...,DR Smith,"Punjab Cricket Association IS Bindra Stadium, ...",A Nand Kishore,VK Sharma,NaN,244,237.500000,183.214286,17.378049,98.653846
4,5,2010,4/17/10,Royal Challengers Bangalore,Mumbai Indians,Royal Challengers Bangalore,field,Normal,0,Mumbai Indians,...,R McLaren,M Chinnaswamy Stadium,HDPK Dharmasena,SJA Taufel,NaN,248,285.166667,420.625000,95.055556,140.208333


In [5]:
data_delivery.head(5)

,Match_ID,Innings,Batting_Team,Bowling_Team,Over,Ball,Batsman,Non_Striker,Bowler,Is_Super_Over,...,Bye_Runs,Legbye_Runs,Noball_Runs,Penalty_Runs,Batsman_Runs,Extras,Total_Runs,Player_Dismissed,Dismissal_Type,Fielder
0,1,1,Pune Warriors India,Delhi Capitals,1,1,RV Uthappa,AJ Finch,IK Pathan,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Pune Warriors India,Delhi Capitals,1,2,RV Uthappa,AJ Finch,IK Pathan,0,...,0,0,0,0,2,0,2,NaN,NaN,NaN
2,1,1,Pune Warriors India,Delhi Capitals,1,3,RV Uthappa,AJ Finch,IK Pathan,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Pune Warriors India,Delhi Capitals,1,4,RV Uthappa,AJ Finch,IK Pathan,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Pune Warriors India,Delhi Capitals,1,5,RV Uthappa,AJ Finch,IK Pathan,0,...,0,0,0,0,1,0,1,NaN,NaN,NaN


## Label Encoding

In [6]:
le = preprocessing.LabelEncoder()

### 1. Teams Encoding

In [7]:
Team1 = list(set(data_match.Team1))
le.fit(Team1)
data_match.Team1 = le.transform(data_match.Team1)
data_match.Team2 = le.transform(data_match.Team2)
data_match.Toss_Winner = le.transform(data_match.Toss_Winner)

### 2. Toss Encoding

In [8]:
toss_decision = list(set(data_match.Toss_Decision))
le.fit(toss_decision)
data_match.Toss_Decision = le.transform(data_match.Toss_Decision)

### 3. Venue Encoding 

In [9]:
venue = list(set(data_match.Venue))
le.fit(venue)
data_match.Venue = le.transform(data_match.Venue)

### 4. Umpire Encoding

In [10]:
s1 = set(data_match.Umpire1)
s2 = set(data_match.Umpire2)
umpire = list(s1.union(s2))
le.fit(umpire)
data_match.Umpire1 = le.transform(data_match.Umpire1)
data_match.Umpire2 = le.transform(data_match.Umpire2)

In [11]:
data_match

,Match_ID,Season,Date,Team1,Team2,Toss_Winner,Toss_Decision,result,DL_Method,Winner,...,Man_Of_The_Match,Venue,Umpire1,Umpire2,Umpire3,Total_Balls,Team1_Strength,Team2_Strength,Team1_Strength.1,Team2_Strength.1
0,1,2013,5/19/13,8,2,8,0,Normal,0,Pune Warriors India,...,LJ Wright,32,31,46,NaN,250,237.500000,242.250000,79.166667,20.471831
1,2,2015,4/15/15,4,2,4,0,Normal,0,Delhi Capitals,...,MA Agarwal,17,14,26,NaN,253,654.750000,1410.500000,100.730769,88.156250
2,3,2011,5/14/11,11,6,11,1,Normal,1,Royal Challengers Bangalore,...,CH Gayle,14,36,35,NaN,159,579.000000,385.375000,28.016129,27.043860
3,4,2017,5/7/17,4,3,3,1,Normal,0,Gujarat Lions,...,DR Smith,22,1,54,NaN,244,237.500000,183.214286,17.378049,98.653846
4,5,2010,4/17/10,11,7,11,1,Normal,0,Mumbai Indians,...,R McLaren,14,19,46,NaN,248,285.166667,420.625000,95.055556,140.208333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,752,2016,5/12/16,12,2,2,1,Normal,0,Delhi Capitals,...,CH Morris,24,24,29,NaN,237,295.416667,433.000000,52.132353,22.789474
752,753,2016,5/25/16,12,6,6,1,Normal,0,Sunrisers Hyderabad,...,MC Henriques,8,29,13,NaN,244,253.214286,188.187500,38.532609,19.551948
753,754,2012,4/5/12,6,2,2,1,Normal,0,Delhi Capitals,...,IK Pathan,7,40,19,NaN,142,398.625000,529.166667,34.663043,38.719512
754,755,2014,5/7/14,4,0,0,1,Normal,0,Kings XI Punjab,...,GJ Maxwell,0,19,33,NaN,248,253.937500,264.000000,25.393750,26.400000


## Feature Engineering

### Creating Day, Month and Day_of_Week 

In [12]:
data_match['Date'] = pd.to_datetime(data_match['Date'], infer_datetime_format=True, errors='coerce')
data_match.dtypes
#Days of week 0-Monday 1-Tuesday 2-Wednesday 3-Thursday 4-Friday 5-Saturday 6-Sunday
data_match['Day_Of_Week'] = data_match['Date'].dt.weekday

#Date format mm dd year
data_match['Month'] = data_match['Date'].dt.month
data_match['Date'] = data_match['Date'].dt.day

### Average Balls Delivered by a team in a season

In [13]:
data = data_delivery.groupby('Match_ID')
result = pd.DataFrame()
for ID, value in data:
    team1 = data_match[data_match.Match_ID == ID].Team1.values[0]
    team2 = data_match[data_match.Match_ID == ID].Team2.values[0]
    team1_balls_delivered = value[value.Bowling_Team == team1].shape[0]
    team2_balls_delivered = value[value.Bowling_Team == team2].shape[0]
    temp = pd.DataFrame({'Team1_Balls_Delivered': team1_balls_delivered, 'Team2_Balls_Delivered': team2_balls_delivered}, index = [ID - 1])
    result = pd.concat([result, temp])

data_match = pd.concat([data_match, result], axis = 1)


In [14]:
result_avg = pd.DataFrame()
for index, row in data_match.iterrows():
    data = data_match[data_match.Season == row.Season]
    apperance = data[data['Date'] <= row.Date]
    team1 = row['Team1']
    team1_total_balls = apperance[apperance.Team1 == team1].shape[0]
    t1 = apperance[apperance.Team1 == team1]
    t1_balls = t1['Team1_Balls_Delivered'].sum()
    t2 = apperance[apperance.Team2 == team1]
    t2_balls = t2['Team2_Balls_Delivered'].sum()
    team1_apperance = pd.concat([t1, t2], axis = 0).shape[0]
    t1_total_balls = t1_balls + t2_balls
    t1_avg_balls = t1_total_balls/team1_apperance
    
    team2 = row['Team2']
    team2_total_balls = apperance[apperance.Team2 == team2].shape[0]
    t1 = apperance[apperance.Team1 == team2]
    t1_balls = t1['Team1_Balls_Delivered'].sum()
    t2 = apperance[apperance.Team2 == team2]
    t2_balls = t2['Team2_Balls_Delivered'].sum()
    team2_apperance = pd.concat([t1, t2], axis = 0).shape[0]
    t2_total_balls = t1_balls + t2_balls
    t2_avg_balls = t2_total_balls/team2_apperance
    
    temp = pd.DataFrame({'Team1_avg_balls':t1_avg_balls, 'Team2_avg_balls':t2_avg_balls}, index = [index])
    result_avg = pd.concat([result_avg, temp], axis = 0)

result_avg

,Team1_avg_balls,Team2_avg_balls
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
...,...,...
751,0.0,0.0
752,0.0,0.0
753,0.0,0.0
754,0.0,0.0


In [15]:
data_match = pd.concat([data_match, result_avg], axis = 1)
data_match = data_match.drop(['Team1_Balls_Delivered','Team2_Balls_Delivered'], axis = 1)

In [16]:
data_match

,Match_ID,Season,Date,Team1,Team2,Toss_Winner,Toss_Decision,result,DL_Method,Winner,...,Umpire3,Total_Balls,Team1_Strength,Team2_Strength,Team1_Strength.1,Team2_Strength.1,Day_Of_Week,Month,Team1_avg_balls,Team2_avg_balls
0,1,2013,19,8,2,8,0,Normal,0,Pune Warriors India,...,NaN,250,237.500000,242.250000,79.166667,20.471831,6,5,0.0,0.0
1,2,2015,15,4,2,4,0,Normal,0,Delhi Capitals,...,NaN,253,654.750000,1410.500000,100.730769,88.156250,2,4,0.0,0.0
2,3,2011,14,11,6,11,1,Normal,1,Royal Challengers Bangalore,...,NaN,159,579.000000,385.375000,28.016129,27.043860,5,5,0.0,0.0
3,4,2017,7,4,3,3,1,Normal,0,Gujarat Lions,...,NaN,244,237.500000,183.214286,17.378049,98.653846,6,5,0.0,0.0
4,5,2010,17,11,7,11,1,Normal,0,Mumbai Indians,...,NaN,248,285.166667,420.625000,95.055556,140.208333,5,4,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,752,2016,12,12,2,2,1,Normal,0,Delhi Capitals,...,NaN,237,295.416667,433.000000,52.132353,22.789474,3,5,0.0,0.0
752,753,2016,25,12,6,6,1,Normal,0,Sunrisers Hyderabad,...,NaN,244,253.214286,188.187500,38.532609,19.551948,2,5,0.0,0.0
753,754,2012,5,6,2,2,1,Normal,0,Delhi Capitals,...,NaN,142,398.625000,529.166667,34.663043,38.719512,3,4,0.0,0.0
754,755,2014,7,4,0,0,1,Normal,0,Kings XI Punjab,...,NaN,248,253.937500,264.000000,25.393750,26.400000,2,5,0.0,0.0


## Dimensionality Reduction

Since there are no 3rd Umpire we are going to drop that column

In [17]:
data_match = data_match.drop(['Umpire3'],  axis = 1)

Since we trying to predict the number of balls, result, Man_Of_The_Match of the match wont play a factor because we cant know the outcome before hand 

In [18]:
data_match = data_match.drop(['result', 'Winner', 'Man_Of_The_Match', 'Win_By_Runs', 'Win_By_Wicket'],  axis = 1)

In [19]:
data_match

,Match_ID,Season,Date,Team1,Team2,Toss_Winner,Toss_Decision,DL_Method,Venue,Umpire1,Umpire2,Total_Balls,Team1_Strength,Team2_Strength,Team1_Strength.1,Team2_Strength.1,Day_Of_Week,Month,Team1_avg_balls,Team2_avg_balls
0,1,2013,19,8,2,8,0,0,32,31,46,250,237.500000,242.250000,79.166667,20.471831,6,5,0.0,0.0
1,2,2015,15,4,2,4,0,0,17,14,26,253,654.750000,1410.500000,100.730769,88.156250,2,4,0.0,0.0
2,3,2011,14,11,6,11,1,1,14,36,35,159,579.000000,385.375000,28.016129,27.043860,5,5,0.0,0.0
3,4,2017,7,4,3,3,1,0,22,1,54,244,237.500000,183.214286,17.378049,98.653846,6,5,0.0,0.0
4,5,2010,17,11,7,11,1,0,14,19,46,248,285.166667,420.625000,95.055556,140.208333,5,4,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,752,2016,12,12,2,2,1,0,24,24,29,237,295.416667,433.000000,52.132353,22.789474,3,5,0.0,0.0
752,753,2016,25,12,6,6,1,0,8,29,13,244,253.214286,188.187500,38.532609,19.551948,2,5,0.0,0.0
753,754,2012,5,6,2,2,1,0,7,40,19,142,398.625000,529.166667,34.663043,38.719512,3,4,0.0,0.0
754,755,2014,7,4,0,0,1,0,0,19,33,248,253.937500,264.000000,25.393750,26.400000,2,5,0.0,0.0


#### Recursive feature elimination

In [20]:
X = data_match.drop(['Total_Balls'], axis = 1)
y = data_match['Total_Balls']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#X_train = X[X.Season != 2019]
#y_train = y[data_match.Season != 2019]

#X_test = X[X.Season == 2019]
#y_test = y[data_match.Season == 2019]

In [21]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
selector = RFE(reg, 5, step=1)
selector = selector.fit(X_train, y_train)
rank = selector.ranking_

index = np.where(rank == 1)
index

(array([ 3,  6,  7, 15, 16]),)

In [22]:
X_train.columns[index]

Index(['Team1', 'Toss_Decision', 'DL_Method', 'Day_Of_Week', 'Month'], dtype='object')

## Prediction Models

In [23]:
X = data_match.drop(['Total_Balls'], axis = 1)
y = data_match['Total_Balls']
X

,Match_ID,Season,Date,Team1,Team2,Toss_Winner,Toss_Decision,DL_Method,Venue,Umpire1,Umpire2,Team1_Strength,Team2_Strength,Team1_Strength.1,Team2_Strength.1,Day_Of_Week,Month,Team1_avg_balls,Team2_avg_balls
0,1,2013,19,8,2,8,0,0,32,31,46,237.500000,242.250000,79.166667,20.471831,6,5,0.0,0.0
1,2,2015,15,4,2,4,0,0,17,14,26,654.750000,1410.500000,100.730769,88.156250,2,4,0.0,0.0
2,3,2011,14,11,6,11,1,1,14,36,35,579.000000,385.375000,28.016129,27.043860,5,5,0.0,0.0
3,4,2017,7,4,3,3,1,0,22,1,54,237.500000,183.214286,17.378049,98.653846,6,5,0.0,0.0
4,5,2010,17,11,7,11,1,0,14,19,46,285.166667,420.625000,95.055556,140.208333,5,4,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,752,2016,12,12,2,2,1,0,24,24,29,295.416667,433.000000,52.132353,22.789474,3,5,0.0,0.0
752,753,2016,25,12,6,6,1,0,8,29,13,253.214286,188.187500,38.532609,19.551948,2,5,0.0,0.0
753,754,2012,5,6,2,2,1,0,7,40,19,398.625000,529.166667,34.663043,38.719512,3,4,0.0,0.0
754,755,2014,7,4,0,0,1,0,0,19,33,253.937500,264.000000,25.393750,26.400000,2,5,0.0,0.0


In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### Linear Regression

In [25]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_train, y_train)
y_predict = reg.predict(X_test)
error = mean_squared_error(y_test, y_predict)
print('MSE of Linear Regression: ', error)
print('RMSE of Linear Regression: ', math.sqrt(error))

MSE of Linear Regression:  207.36085067782142
RMSE of Linear Regression:  14.400029537394062


### Support Vector Regression

In [26]:
def SVR(X_train, y_train):
    from sklearn.svm import SVR
    clf = SVR(kernel = 'poly', gamma='scale', C=1.0, epsilon=0.2)
    fit = clf.fit(X_train, y_train)
    
    return fit

In [27]:
SVR_fit = SVR(X_train, y_train)
y_predict = SVR_fit.predict(X_test)
error = mean_squared_error(y_test, y_predict)
print('MSE of SVR: ',error)
print('RMSE of SVR: ',math.sqrt(error))

MSE of SVR:  250.961948267336
RMSE of SVR:  15.841778570202779


### SVR using Stocastic Gradient Decent  

In [28]:
def svr_SGD_epsilion_insensitive(X_train, y_train):
    from sklearn.linear_model import SGDRegressor
    svr_SGD = SGDRegressor(loss = 'epsilon_insensitive' , max_iter = 1000 , tol = 1e-3 , shuffle = True , epsilon = .1 , learning_rate = 'adaptive')
    y_SGD = svr_SGD.fit(X_train, y_train)
    
    return svr_SGD

In [29]:
reg = svr_SGD_epsilion_insensitive(X_train, y_train)
y_predict = reg.predict(X_test)
error = mean_squared_error(y_test, y_predict)
print('MSE of SGD Epsilion Insensitive: ',error)
print('RMSE of SGD Epsilion Insensitive: ',math.sqrt(error))

MSE of SGD Epsilion Insensitive:  5986.747390221051
RMSE of SGD Epsilion Insensitive:  77.37407440623151


### Logistic Regression

In [30]:
from sklearn.linear_model import LogisticRegression

reg = LogisticRegression(random_state=0, solver='saga', multi_class='multinomial', max_iter = 10000)
fit = reg.fit(X, y)
y_predict = fit.predict(X_test)

error = mean_squared_error(y_test, y_predict)
print('MSE of Neural Network: ',error)
print('MSE of Neural Network: ',math.sqrt(error))


MSE of Neural Network:  252.72246696035242
MSE of Neural Network:  15.897247150382748
